In [13]:
using DynamicPolynomials, LinearAlgebra

@ncpolyvar x1 x2 x3 y1 y2 y3

var=Vector{PolyVar{false}}([x1;x2;x3;y1;y2;y3])

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function get_star_alg(f)
    mom=monomials(f)
    coe=coefficients(f)
    mom=star_algebra.(mom)
    return dot(mom,coe)
end



u = x1*(y1+y2+y3)+x2*(y1+y2-y3)+x3*(y1-y2)-x1-2.0*y1-y2

f = -(u + get_star_alg(u))/2

g=Vector{Polynomial{false,Float64}}([])

h=1.0*[x1-x1^2;
    x2-x2^2;
    x3-x3^2;
    y1-y1^2;
    y2-y2^2;
    y3-y3^2;
    x1*y1-y1*x1;
    x2*y1-y1*x2;
    x3*y1-y1*x3;
    x1*y2-y2*x1;
    x2*y2-y2*x2;
    x3*y2-y2*x3;
    x1*y3-y3*x1;
    x2*y3-y3*x2;
    x3*y3-y3*x3];

f=f([x1;x2;x3;y1;y2;y3]=>[0.5*x1+0.5;0.5*x2+0.5;0.5*x3+0.5;0.5*y1+0.5;0.5*y2+0.5;0.5*y3+0.5])

for j in 1:length(h)
    h[j]=4.0*h[j]([x1;x2;x3;y1;y2;y3]=>[0.5*x1+0.5;0.5*x2+0.5;0.5*x3+0.5;0.5*y1+0.5;0.5*y2+0.5;0.5*y3+0.5])
end

In [14]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(var,f,g,h);

In [15]:
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

Number of variables: n=6
Number of inequality constraints: m=0
Number of equality constraints: l=15


In [32]:
k=2

println("Relaxation order: k=",k)

Relaxation order: k=2


In [33]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-3,
                                 use_eqcons_to_get_constant_trace=true,
                                 check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 43.0
  Number of blocks: omega=1
  Size of the largest block: s^max=43
  Number of equality trace constraints: zeta=484
Modeling time:
  0.198322 seconds (296.58 k allocations: 23.548 MiB)
iter=1   val=-10.750000000000004   gap=9.07075832885234e-16   feas=86.45253996654276
iter=2   val=-1.866665189630151   gap=0.8283195941241884   feas=19.94290130761034
iter=4   val=-0.5688001080318088   gap=0.8361665031497935   feas=11.749674831877117
iter=8   val=-1.9916905797640183   gap=0.5861351366487483   feas=19.53042675133582
iter=16   val=-0.30418565828481975   gap=0.7766625243653403   feas=4.189626238550077
iter=32   val=-0.17419331105640434   gap=0.5170362050217718   feas=4.834260782020131
iter=64   val=-0.2914470682307424   gap=0.3194251410865663   feas=1.3014470061046566
iter=128   val=-0.22431016599984113   gap=0.3131466208609536   feas=0.6369393095932843
iter=256   val=-0.25278394807590465   gap=0.2491406090256964   feas=0

-0.2508968294997016

In [34]:
using NCTSSOS

@time ctpNCPOP.nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
    n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS=false,QUIET=false);

------------------------------------------------------
The sizes of blocks:
[43]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 820             
  Cones                  : 0               
  Scalar variables       : 421             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - numb